<a href="https://www.kaggle.com/code/gabripo93/the-perfect-match-for-your-tech-and-business-needs?scriptVersionId=209753307" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

# In-chat Multiagents to Find the Right Company and Generate Clause-by-Clause Reports for Tenders 📑💼

The following Kaggle notebook takes advantage of Gemini's long context window to achieve the following objectives:

- Analyze technical and commercial tenders for a project. 📊
- Assess the compatibility of companies' products and solutions with tender documents. 🔍
- Identify the best company and product-service combination to execute the project, generating a clause-by-clause report with compliant and non-compliant specifications. 📋✅❌

## Notebook Structure 📓

The notebook is divided into different sections, each with a specific objective:

- Dataset load (see *Relevant Project and Open points* chapter for data generation)
- Tenders for a project are parsed, converting their information into text. 📝
- Information scraped from various companies' websites is loaded as text. 🕸️
- All text is processed by Gemini using different prompts, combining multi-agent reasoning, chain of thoughts, and in-chat memory. 🤖💭

## Multi-agent Reasoning 🧠

Multi-agent reasoning is implemented by segmenting tasks and delegating responsibilities to distinct roles. For example:

- **Technical and Commercial Tender Agents**: Separate prompts (tender_prompt_template_technical and tender_prompt_template_commercial) guide the roles of the technical tender engineer and commercial tender manager. Each agent has distinct objectives: identifying and summarizing technical or commercial requirements within tenders. This multi-agent structure ensures detailed and domain-specific analyses. 👷‍♂️💼

- A distinct prompt is also prepared for analyzing companies (e.g., SIEMENS and HITACHI) to match tender requirements with their products and solutions (get_response_companies_info). This allows tailored reasoning for comparing affinities between tenders and company offerings. 🏢🔄

## Chain of Thoughts 🧩

The chain of thoughts approach decomposes complex tasks into sequential, step-by-step actions, ensuring methodical problem-solving. In both technical and commercial prompts, we used phrases like "Think step by step" to guide the agent toward incremental reasoning. This ensures that requirements are dissected and analyzed in detail. 🔍🧠

The user prompt specifies a structured approach to calculating an affinity score, prompting the agent to explicitly explain the calculation process. Finally, in the Clause-by-Clause Analysis, the final prompt directs the agent to meticulously compare tender requirements with company specifications, maintaining a clear progression in thought. This approach is embedded in the tender query processing and the affinity scoring logic in user_prompt_match and final_prompt, encouraging logical progression in the analysis. 📈🔗

## In-chat Memory 🗃️

The code uses in-chat memory to maintain conversational context across multiple interactions.

In-chat memory stores all the interactions from the technical and commercial tender analysis, keeping track of the responses from the different roles (e.g., technical engineer, commercial manager, sales manager). This memory allows to build upon the context of earlier prompts without having to constantly reprocess the same information. With context caching the system stores intermediate results from prior tender evaluations or company analyses, so if a similar query arises, the system can quickly retrieve relevant data and produce faster, more accurate responses.

This functionality is facilitated by:

- **Chat History Preservation**: The function add_history_to_chat appends user queries and model responses (e.g., for tenders or company analyses) to history_chat. This ensures continuity, enabling the model to refer back to previous inputs and outputs during subsequent exchanges. 📝🔄

- Prompts such as system_prompt and user_prompt leverage the accumulated chat history to enhance the depth and relevance of responses. For example, when computing affinity scores or performing a clause-by-clause analysis, the model can reference earlier content in the chat_with_memory object. This allows continuous improvement of the prompt and on the information stored in the chat. 🗣️🔍

## Conclusion for the Use Case 🤔

Using a long context window instead of Retrieval-Augmented Generation (RAG) for this use case was particularly beneficial due to the task's nature, which involves reasoning across interdependent documents while maintaining conversational continuity and ensuring consistent context for decision-making. The unified context allows the model to cross-reference tender requirements and company offerings directly, ensuring cohesive and accurate analysis. This is particularly advantageous for tasks like affinity scoring, which require simultaneous consideration of multiple data points. 📊🔗

The notebook's approach scales better for handling multiple queries simultaneously, as it avoids the bottleneck of sequential agent calls. For new tender projects, it's only necessary to update the in-chat memory and add new prompts for adding new in-chat agents. 🔄🔄

In summary, why did we decide to build this notebook?

1. **Holistic Context Retention** 📚: By storing the entire history of tender analyses (both technical and commercial) and company product evaluations, the model retains a comprehensive understanding of all previously provided information. This holistic context allows the model to reason about how specific requirements and offerings interrelate across multiple prompts. In RAG, the system retrieves only the most relevant chunks of information for each query; this efficient approach can lead to fragmented analyses, potentially overlooking interconnections.

2. **Interdependent Analysis** 🔄: This task involves comparing multiple tenders against products and solutions offered by different companies, followed by calculating an affinity score and conducting a clause-by-clause compliance analysis. These steps require accessible and integrated information from previous steps. RAG typically retrieves context independently for each query, which might result in a loss of nuance or context-dependent reasoning, especially when relationships between multiple documents must be preserved. A long context window ensures the model has immediate access to the entire conversational flow and insights developed so far.

3. **Dynamic Multi-Agent Collaboration** 🤝: By maintaining a long context, the system can simulate multi-agent collaboration, allowing outputs from technical engineers, commercial managers, and sales managers to flow into a unified reasoning framework. In RAG, each role’s analysis would require re-retrieving relevant information, possibly leading to inconsistencies or duplications. A long context window naturally informs each role, creating a seamless chain of thought.

4. **Reduced Query Overhead** 🔄: Long context windows reduce the need for multiple retrieval calls, making the process more efficient in scenarios where information is revisited or refined iteratively. RAG introduces latency and computational costs because each query requires searching and ranking document chunks. A long context window allows for continuous focus on the task, with all prior exchanges readily available.

5. **Affinity Score Calculation** 📈: Computing an affinity score across companies for tenders requires integrating technical and commercial analysis alongside company data. This step benefits significantly from the model's ability to access all previous responses simultaneously. In RAG, affinity scoring would require separate retrievals of technical requirements, commercial requirements, and company data for each tender. This could introduce discrepancies if context for one query is inadvertently excluded during retrieval.

6. **Clause-by-Clause Compliance Analysis** 🔄: Clause-by-clause analysis relies on cross-referencing previously extracted requirements with company offerings. The long context window allows the model to directly reference earlier inputs and outputs without reloading or retrieving. RAG retrievals for clause-by-clause analysis might lead to inconsistencies if prior reasoning is split across multiple retrievals. A long context window ensures the model "remembers" and applies earlier analyses cohesively.

### Related Projects and Open Points 📁

The data generation and cleaning is performed with another repo stored in github:  https://github.com/gabripo/kaggle-gemini-long-context.

In the past few months, we also implemented a multi-agent framework (LumadaAI) using LangChain and OpenAI, where each company was represented by a dedicated agent. **LumadaAI** is publicly available at https://github.com/SecchiAlessandro/LumadaAI. This framework featured a supervisor agent that dynamically routed user queries to the most relevant company-specific agent based on the query context. While innovative, this approach faced challenges in stability, accuracy, and efficiency, making the current solution more effective. As agents operated independently, generating combined solutions from different companies was difficult. Additionally, for each query, the supervisor needed to perform additional reasoning before invoking an agent. If a query was relevant to multiple agents, the framework had to perform sequential calls, compounding latency. The current solution with centralized reasoning ensures consistent application of logic and context. By avoiding the intermediate step of agent selection, it directly processes queries with unified context, reducing latency significantly. 

**EasyRAG** (https://github.com/gabripo/easyrag) is another RAG tool that performs RAG over locally stored documents. We are benchmarking this tool with Gemini's long context window: adding one or more PDFs to Gemini's context window could provide more precise insights than the RAG approach. 📈

## Conclusion 🔽

The centralized, long context window approach provides clear advantages in stability, response time, and accuracy over the earlier multi-agent framework. It highlights the importance of selecting a system architecture that aligns with the specific demands of the use case, particularly for complex, multi-faceted analyses like those in tender evaluations, clause-by-clause generation, and company affinity scoring. 📊🔗

#### The long context window acts as a shared workspace, recording and making all agent outputs accessible for seamless and holistic reasoning. In today's interconnected world, where partnerships and synergies are essential to addressing complex challenges, we envision a tool that enables continuous reasoning, uncovers new patterns and solutions, and minimizes the fragmentation of insights. 🌐🔍💡



In [1]:
# import Python libraries
import os
import json
from IPython.display import Markdown

In [2]:
# auxiliary function to read JSON files
def read_json_info(jsonFilePath: str) -> dict:
    if os.path.exists(jsonFilePath):
        with open(jsonFilePath, "r") as f:
            data = json.load(f)
        return data
    else:
        return {}

In [3]:
# auxiliary Python decorator to execute a function again, if its execution fails
# this is helpful when calling the Gemini's API since Gemini has a rate limiter and, if an execution fails for that, there will be some waiting time before retrying
import time

def retry_on_failure(wait_time_seconds=60, max_retries=5):
    def decorator_retry(func):
        
        def wrapper_retry(*args, **kwargs):
            retries = 0
            while retries < max_retries:
                try:
                    return func(*args, **kwargs)
                except Exception as e:
                    retries += 1
                    if retries < max_retries:
                        print(
                            f"Function failed with error: {e}. Retrying in {wait_time_seconds} seconds... (Attempt {retries}/{max_retries})"
                        )
                        time.sleep(wait_time_seconds)
                    else:
                        print(f"Function failed after {max_retries} attempts.")
                        raise e
        return wrapper_retry

    return decorator_retry

In [4]:
dataset_path = '/kaggle/input/tenders-and-companies-websites'
working_path = '/kaggle/working'

In [5]:
!mkdir -p /kaggle/working/tenders
tenders_working_path = os.path.join(working_path, 'tenders')

!mkdir -p /kaggle/working/companies
companies_working_path = os.path.join(working_path, 'companies')

# Build a chat with Gemini

In [6]:
# API key got here: https://ai.google.dev/tutorials/setup

import google.generativeai as genai
from kaggle_secrets import UserSecretsClient


user_secrets = UserSecretsClient()
secret_key = user_secrets.get_secret("GEMINI_API_KEY")

genai.configure(api_key = secret_key)

model_name = 'gemini-1.5-flash-latest'
model = genai.GenerativeModel(model_name=model_name)

model_info = genai.get_model(f"models/{model_name}")
print(f"{model_info.input_token_limit=}")
print(f"{model_info.output_token_limit=}")

model_info.input_token_limit=1000000
model_info.output_token_limit=8192


In [ ]:
print("List of models that support generateContent:\n")
for m in genai.list_models():
    if "generateContent" in m.supported_generation_methods:
        print(m.name)

In [7]:
# the decorator ensures that, if an error occurs, the function will be executed again
@retry_on_failure(wait_time_seconds=60, max_retries=5)
def ask_gemini(prompt, chat_with_memory=None, model=None, history=[], model_name = 'gemini-1.5-flash-latest'):
    """
    function to call Gemini, providing chat history
    if a chat is already available, it will be used
    """
    if model == None:
        model = genai.GenerativeModel(model_name=model_name)
        
    if chat_with_memory == None:
        # since no chat is already available, create a new one
        chat_with_memory = model.start_chat(history=history)
    
    response = chat_with_memory.send_message(prompt)
    return response, chat_with_memory

# Analyze the tenders

In [8]:
# read the json file related to tenders from the input dataset
tenders_info_json_path = os.path.join(dataset_path, 'tenders_info.json')
tenders_info = read_json_info(tenders_info_json_path)

# tenders_info is a dictionary, where the key is the name of the tender file and the related value its information
# print(tenders_info["tender_wind.pdf"])

In [9]:
# list the processed tender files
tenders = tenders_info.keys()

In [10]:
tender_prompt_template_technical = """
You are an experienced technical tender engineer. 
The document you have is a tender, that contains also technical requirements for a project.
Think step by step on how to look for the relevant technical requirements and make a detailed summary.
The content of the document is: """
tender_prompts_technical = []
for info in tenders_info.values():
    tender_prompts_technical.append(f"You have a document called {info['name']} . " + tender_prompt_template_technical + f"{info['content']}")

In [11]:
tender_prompt_template_commercial = """
You are an experienced commercial tender manager. 
The document you have is a tender, that contains also commercial requirements for a project.
Think step by step on how to look for the relevant commercial requirements and make a detailed summary.
The content of the document is: "
"""
tender_prompts_commercial = []
for info in tenders_info.values():
    tender_prompts_commercial.append(f"You have a document called {info['name']} . " + tender_prompt_template_commercial + f"{info['content']}")

In [12]:
@retry_on_failure(wait_time_seconds=60)
def get_responses_tenders(subject, tender_prompts):
    tenders_json_file_path = os.path.join(tenders_working_path, f'tenders_{subject}.json')
    
    if os.path.exists(tenders_json_file_path):
        responses = read_json_info(tenders_json_file_path)
        print(f"tender_{subject}: Responses loaded from file {tenders_json_file_path}")
    else:
        for tender_prompt, tender_name in zip(tender_prompts, tenders):
            print(f"tender_{subject}: Generating response for tender {tender_name} ...")
            response, _ = ask_gemini(prompt = tender_prompt)
            #print(response.text)

            responses = {}
            responses[tender_name] = {'prompt': tender_prompt, 'answer': response.text}
            print(f"tender_{subject}: Response for tender {tender_name} generated.")
    
        with open(tenders_json_file_path, 'w') as f:
            json.dump(responses, f, ensure_ascii=True, indent=4)
        print(f"tender_{subject}: Responses stored into {tenders_json_file_path}")
    
    print(f"tender_{subject}: Analysis concluded!\n")
    return responses

# each call of get_responses_tenders() will generate a tenders_{subject}.json file
# each generated file so will contain the Gemnini's responses for a given subject
response_technical = get_responses_tenders("technical", tender_prompts_technical)
response_commercial = get_responses_tenders("commercial", tender_prompts_commercial)

tender_technical: Generating response for tender tender_wind.pdf ...
tender_technical: Response for tender tender_wind.pdf generated.
tender_technical: Generating response for tender tender_solar.pdf ...
tender_technical: Response for tender tender_solar.pdf generated.
tender_technical: Responses stored into /kaggle/working/tenders/tenders_technical.json
tender_technical: Analysis concluded!

tender_commercial: Generating response for tender tender_wind.pdf ...
tender_commercial: Response for tender tender_wind.pdf generated.
tender_commercial: Generating response for tender tender_solar.pdf ...
tender_commercial: Response for tender tender_solar.pdf generated.
tender_commercial: Responses stored into /kaggle/working/tenders/tenders_commercial.json
tender_commercial: Analysis concluded!



# Analyze the companies products and solutions

## Data generation
Information about interesting companies is obtained from their websites.

To generate data out of the companies' websites, we implemented a crawler.

The final output of the crawler is a JSON file, in which each field refers to a company: for each company, all the information of the websites is merged.

> The generation of information can be found in the Kaggle Notebook https://www.kaggle.com/code/gabripo93/gemini15-long-context-competition-generate-dataset

### Details about the crawling process:
- **Recursive scan**: after a webpage is scanned and its content is stored, eventual found sublinks are scanned, as well. A limit of the wepages to download is given as input.
- **Redundant information is deleted**: if some website content can be found multiple times in all the webpages of one company, then it is skipped. *Example*: undesired and redundant lines like "Contact Us" are removed, ensuring that the final content does not include unnecessary sentences.
- **Caching of already downloaded pages**: for each webpage, the content is stored in a JSON file, as well as the found sublinks. *Example*: after a run with a limit of N pages, other runs with less than N pages will use the stored files instead downloading data from internet; at the contrary, if the limit is increased to M > N pages, only M - N additional pages will be downloaded while the first N pages will be taken from the stored file. 

In [13]:
companies_info_json_path = os.path.join(dataset_path, 'companies_info.json')
companies_info = read_json_info(companies_info_json_path)

# companies_info is a dictionary, where the key is the name of the company and the related value its information
# print(companies_info["SIEMENS"]) 

In [14]:
@retry_on_failure(wait_time_seconds=60)
def get_response_companies(company_name):
    companies_json_file_path = os.path.join(companies_working_path, f'companies_{company_name}.json')
    
    if os.path.exists(companies_json_file_path):
        responses = read_json_info(companies_json_file_path)
        print(f"companies_{company_name}: Responses loaded from file {companies_json_file_path}")
    else:
        print(f"companies_{company_name}: Generating response for company {company_name} ...")
        responses = {}
        company_prompt = f"These are the information of products and solutions for the company {company_name} : {companies_info[company_name]}"
        response, _ = ask_gemini(prompt = company_prompt)
        responses[company_name] = {'prompt': company_prompt, 'answer': response.text}

        with open(companies_json_file_path, 'w') as f:
                json.dump(responses, f, ensure_ascii=True, indent=4)
        print(f"companies_{company_name}: Responses stored into {companies_json_file_path}")

    print(f"companies_{company_name}: Response for company {company_name} generated!")
    return responses

# each call of get_responses_companies() will generate a companies_{company_name}.json file
# each generated file so will contain the Gemnini's responses for a given company
# the purpose of generating responses given companies information is to store it in the chat history


In [15]:
response_hitachi = get_response_companies("HITACHI")

companies_HITACHI: Generating response for company HITACHI ...
companies_HITACHI: Responses stored into /kaggle/working/companies/companies_HITACHI.json
companies_HITACHI: Response for company HITACHI generated!


In [16]:
response_siemens = get_response_companies("SIEMENS")

companies_SIEMENS: Generating response for company SIEMENS ...
companies_SIEMENS: Responses stored into /kaggle/working/companies/companies_SIEMENS.json
companies_SIEMENS: Response for company SIEMENS generated!


# Build a chat history based on previous prompts

In [17]:
# example how to include the chat history here https://github.com/GoogleCloudPlatform/generative-ai/blob/main/gemini/getting-started/intro_gemini_chat.ipynb
# description of the Content class here https://github.com/google-gemini/generative-ai-python/blob/main/docs/api/google/generativeai/GenerativeModel.md
from google.generativeai.protos import Content, Part

history_chat = []

def add_history_to_chat_single(response, user, history_chat):
    query = Part()
    query.text = f"{user}: {response['prompt']}"
    history_chat.append(Content(role="user", parts=[query]))

    answer = Part()
    answer.text = response['answer']
    history_chat.append(Content(role="model", parts=[answer]))
    return

def add_history_to_chat(responses, user, history_chat):
    for response in responses.values():
        add_history_to_chat_single(response, user, history_chat)
    return 

add_history_to_chat(response_technical, "technical engineer", history_chat)
add_history_to_chat(response_commercial, "commercial manager", history_chat)
add_history_to_chat(response_siemens, "sales manager for siemens", history_chat)
add_history_to_chat(response_hitachi, "sales manager for hitachi", history_chat)

## Test the chat history

In [18]:
prompt_roles = "Which are the roles given in the prompt from the user? There are only two for tenders and one for company"
response_roles, gemini_chat = ask_gemini(prompt=prompt_roles, history=history_chat)

Markdown(response_roles.text)

Function failed with error: 429 Resource has been exhausted (e.g. check quota).. Retrying in 60 seconds... (Attempt 1/5)


The prompt mentions three roles:

1. **Technical Tender Engineer:** This role focuses on evaluating the technical aspects of a tender document, identifying and summarizing the relevant technical requirements for a project.

2. **Commercial Tender Manager:** This role centers on the commercial aspects of a tender, analyzing the financial implications, pricing strategies, and contractual requirements.

3. **Sales Manager (for Siemens/Hitachi):** This is a company-specific role, focusing on understanding a company's product portfolio and presenting relevant products and services to meet a specific tender's requirements.  The prompt uses this role to assess the company's capabilities in responding to a tender.


In [19]:
# add the last response to the chat history
add_history_to_chat_single({'prompt': prompt_roles, 'answer': response_roles.text}, "technical engineer", history_chat)

# Find the most suitable company

In [20]:
prompt_match = """

1. For companies SIEMENS and HITACHI, find the relevant products and solutions with respect to the analyzed tenders. The information is in the form of text I provided, then you do not need to read additional documents or access to websites.
   
2. Calculate an affinity score in percentage for each company based on analysis in point 1 . Explain the way how you computed the affinity score. When possible, use tables and other effective representation ways to summrize numbers and specific information.

"""

In [21]:
print("Finding the most suitable company for the tenders ...")
response_match, gemini_chat = ask_gemini(prompt=prompt_match, chat_with_memory=gemini_chat)
print("Response to the prompts is ready!")

Markdown(response_match.text)

Finding the most suitable company for the tenders ...
Function failed with error: 429 Resource has been exhausted (e.g. check quota).. Retrying in 60 seconds... (Attempt 1/5)
Function failed with error: 429 Resource has been exhausted (e.g. check quota).. Retrying in 60 seconds... (Attempt 2/5)
Function failed with error: 429 Resource has been exhausted (e.g. check quota).. Retrying in 60 seconds... (Attempt 3/5)
Function failed with error: 429 Resource has been exhausted (e.g. check quota).. Retrying in 60 seconds... (Attempt 4/5)
Response to the prompts is ready!


To effectively analyze Siemens and Hitachi's suitability for the Barclayville Solar Power Plant tender, we need to revisit the tender's requirements.  The original prompt provided extensive product information for both companies but did not explicitly list all the *commercial* requirements of the tender itself.  Therefore, I will focus on the *technical* aspects outlined in the initial prompt and  assess the product overlap.  A truly accurate commercial affinity score would require a full commercial specification from the tender document.

**Step 1: Relevant Products and Solutions per Company**

Based solely on the technical requirements in the original prompt and the provided product descriptions for Siemens and Hitachi, we can identify the following relevant products:


**Barclayville Solar Power Plant Technical Requirements & Matching Products:**

| Tender Item No. | Requirement Description                                                                     | Siemens Matching Products/Solutions                                                        | Hitachi Matching Products/Solutions                                      |
|-----------------|---------------------------------------------------------------------------------------------|------------------------------------------------------------------------------------|--------------------------------------------------------------------|
| 1               | Solar PV Array (200 kWp)                                                                   | Not directly specified in provided text.                                               | Not directly specified in provided text.                            |
| 2               | PV Mounting Structure (200 kWp)                                                             | Not directly specified in provided text.                                               | Not directly specified in provided text.                            |
| 3               | Inverter (140 kW)                                                                        | Not directly specified in provided text.                                               | Not directly specified in provided text.                            |
| 4               | Battery (400 kWh)                                                                         | Not directly specified in provided text.                                               | Not directly specified in provided text.                            |
| 5               | Diesel Genset (180 kW/225 kVA)                                                             | SGT-50 (if suitable for this power output), other models might apply                        | Not directly specified in provided text.                            |
| 6               | Electrical BOS (cables, junction boxes)                                                     | Various components mentioned, but no specific product lines for this complete system  | Not directly specified in provided text.                            |
| 7               | Powerhouse buildings and parking                                                          | Not a product offering, but engineering expertise is implied.                                | Not a product offering, but engineering expertise is implied.        |
| 8               | Installation labor, tools, equipment                                                       | Services offered, but not explicitly detailed.                                           | Services offered, but not explicitly detailed.                       |
| 9               | Monitoring System (SCADA)                                                               | Omnivise Asset Management                                                          | Not directly specified in provided text.                            |
| 10              | Warranty                                                                                  | Services mentioned, but not detailed in terms of offered warranties.                        | Services mentioned, but not detailed in terms of offered warranties.        |
| 11              | Manuals                                                                                   | Services offered, but no specific manual products mentioned.                                | Services offered, but no specific manual products mentioned.            |
| 12              | Substation (0.4/11 kV, 500 kVA)                                                            | High-voltage substations (if specifications match), no specific kVA rating mentioned | Not specified in detail; this requires more information from Hitachi. |
| 13-22           | Mini-grid infrastructure (cables, transformers, customer connections, SHS)                 | Various components mentioned, but not a complete mini-grid system.                     | Various components mentioned, but not a complete mini-grid system.     |
| 23              | Spare parts                                                                              | Services offered.                                                                  | Services offered.                                                 |
| 24-25           | Training and After-sales Services                                                          | Services offered.                                                                  | Services offered.                                                 |


**Step 2: Affinity Score Calculation**

Calculating a precise affinity score requires a weighted scoring system based on the *relative importance* of each tender item.  Since this information is missing, we'll use a simplified approach:

**Simplified Affinity Score:**

We will assign a score of "1" if a company has a directly relevant product or service for a given tender item, and "0" if not.  The affinity score is then the sum of these scores, divided by the total number of tender items (25), and expressed as a percentage.


| Company     | Item 1 | Item 2 | Item 3 | Item 4 | Item 5 | Item 6 | Item 7 | Item 8 | Item 9 | Item 10 | Item 11 | Item 12 | Item 13-22 | Item 23 | Item 24-25 | Total Score | Affinity Score (%) |
|-------------|--------|--------|--------|--------|--------|--------|--------|--------|--------|---------|---------|---------|------------|---------|------------|-------------|--------------------|
| Siemens     | 0      | 0      | 0      | 0      | 1      | 0      | 0      | 0      | 1      | 0       | 0       | 0       | 0          | 1      | 1          | 4            | 16%                 |
| Hitachi     | 0      | 0      | 0      | 0      | 0      | 0      | 0      | 0      | 0      | 0       | 0       | 0       | 0          | 1      | 1          | 2            | 8%                  |


**Explanation of Calculation:**

The total possible score is 25 (one point for each of the 25 tender items). Siemens received a total score of 4 and Hitachi received 2, meaning that Siemens has a higher affinity score according to our simplified calculation.


**Limitations:**

This simplified approach significantly underestimates the complexity of tender evaluation.  A true affinity score would necessitate:

* **Weighted Scoring:**  Assigning weights to each tender item based on its importance in the overall evaluation criteria.
* **Detailed Specification Comparison:**  A point-by-point comparison of Siemens and Hitachi's offered specifications against the detailed requirements of each tender item.
* **Commercial Evaluation:**  Assessing commercial aspects like pricing, warranty terms, and after-sales service offerings.
* **Risk Assessment:**  Considering potential risks associated with each company's proposal.


This analysis provides a *preliminary* indication of relative suitability based only on available limited data.  A thorough tender evaluation would require significantly more detail.


In [22]:
# add the last response to the chat history
add_history_to_chat_single({'prompt': prompt_match, 'answer': response_match.text}, "technical engineer", history_chat)

# Generate the clause-by-clause

In [23]:
user_prompt = """

Consider the company with the highest affinity score 
and return the clause by clause analysis considering technical and commercial compliant and not-compliant requirements
of the tender with respect to the selected company. For each company, report the URL of the source where you found information about mentioned products and solutions.
When possible, show data and explain the reasons behind your thinking in tables.

"""

In [24]:
system_prompt = """
You are an experienced team of business development managers and tender engineers, commercial managers.
You need to create a detailed clause by clause from the tender documentations and the most affine company specifications.
"""

In [25]:
print("Generating the clause by clause ...")
prompt_clause_by_clause = f"{system_prompt} {user_prompt}"
response_clause_by_clause, gemini_chat = ask_gemini(prompt=prompt_clause_by_clause, chat_with_memory=gemini_chat)
print("Response to the prompts is ready!")

Markdown(response_clause_by_clause.text)

Generating the clause by clause ...
Function failed with error: 429 Resource has been exhausted (e.g. check quota).. Retrying in 60 seconds... (Attempt 1/5)
Response to the prompts is ready!


Because the original prompt lacked complete commercial specifications for the tender, this analysis will focus primarily on the *technical* aspects and Siemens' capabilities, as determined by the previous analysis. A full commercial clause-by-clause analysis would require the complete tender document including commercial requirements, pricing details, payment terms, and contractual obligations.  Additionally, the provided text did not always include full specifications for Siemens' products – therefore, assumptions are necessary.

**Company Selected:** Siemens (higher affinity score in previous analysis)

**Note:**  The URLs for Siemens' product pages are provided where relevant in the previous response and will not be repeated here for brevity.  The following assumes Siemens can provide suitable solutions,  even if not explicitly detailed in the provided text snippets.  A real-world tender response would require complete data sheets and specifications for all proposed equipment.

**Clause-by-Clause Analysis (Siemens):**

This analysis uses the numbered tender items (1-23) as a basis for clauses.  It assumes that subsequent sections (24, 25: Training and After-Sales Service) are covered by general contractual clauses to be negotiated.

| Tender Item No. | Requirement Description                                                                     | Siemens Compliant? | Siemens Solution                                                                 | Compliance Explanation/Non-Compliance Reason                                                                      |
|-----------------|---------------------------------------------------------------------------------------------|--------------------|---------------------------------------------------------------------------------|-----------------------------------------------------------------------------------------------------------------|
| 1               | Solar PV Array (200 kWp), specific module requirements (technology, power, warranty, standards) | Partially Compliant |  Needs detailed proposal specifying compliant PV modules meeting all requirements. | Requires selecting specific models meeting the 25-year warranty, power output, and certification requirements.   |
| 2               | PV Mounting Structure (200 kWp),  material & dimensional requirements                        | Partially Compliant | Needs detailed proposal outlining a compliant structure.                           | Must specify materials, dimensions, wind rating, and galvanization details according to tender specifications.        |
| 3               | Inverter (140 kW), efficiency, protection, communication features                           | Partially Compliant | Needs detailed proposal outlining a compliant inverter.                          | Must specify exact model meeting all efficiency, protection, and communication requirements.                   |
| 4               | Battery (400 kWh), Lithium-ion, performance and warranty requirements                         | Partially Compliant | Needs detailed proposal specifying compliant battery system.                      |  Requires specifying a Lithium-ion battery model that meets all performance (cycle life, self-discharge), and warranty requirements. |
| 5               | Diesel Genset (180 kW/225 kVA), features and standards                                        | Partially Compliant | SGT-50 (if power output matches); other models may be suitable.                     |  Needs verification that selected model meets all specifications (cold starting, smart control, monitoring, standards). |
| 6               | Electrical BOS (cables, junction boxes), installation requirements                        | Partially Compliant | Requires a detailed proposal detailing compliant cables and junction boxes.        | Must specify cable types (UL 4703 equivalent), junction box types, and installation methods that adhere to tender specs. |
| 7               | Powerhouse buildings and parking                                                          | Not Applicable     | This is a construction requirement, not a product/equipment specification.             |                                                                                                                   |
| 8               | Installation labor, tools, equipment                                                       | Compliant          |  Siemens would propose its installation services in the tender response.           | This is a service offering typically detailed in the tender response.                                           |
| 9               | Monitoring System (SCADA)                                                                | Compliant          | Omnivise Asset Management (if functionality meets all requirements)             |  Must demonstrate that the proposed system satisfies all data logging, reporting, and remote access requirements.     |
| 10              | Warranty (minimum 2 years)                                                              | Partially Compliant | Requires a specific warranty proposal.                                           |  Needs to detail the warranty period, coverage, and procedure.                                                |
| 11              | Manuals (installation, maintenance, troubleshooting)                                         | Compliant          | Siemens would supply manuals as part of the tender submission.                  | Manuals must be in English and meet the specified format and content.                                         |
| 12              | Substation (0.4/11 kV, 500 kVA)                                                            | Partially Compliant |  Siemens would propose a solution, needs detailed specification.                   | Needs to specify exact substation design to meet the 500 kVA capacity and other requirements.                    |
| 13-23           | Mini-grid infrastructure (cables, transformers, customer connections, SHS), spare parts       | Partially Compliant | Requires a detailed proposal specifying compliant components.                     | Requires selecting specific components and quantities to fulfill all technical requirements of the mini-grid.      |


**Commercial Considerations (Missing from Prompt):**

The provided text snippets do not include the full commercial terms of the tender.  A complete response would require addressing the following:

* **Pricing:** Detailed breakdown of costs for all equipment, materials, labor, transportation, and services.
* **Payment terms:** Schedule and method of payments.
* **Contract duration:**  Timeline for project completion.
* **Risk allocation:**  Clearly defined responsibilities and liabilities of both parties.
* **Insurance:**  Requirements for various insurance policies.
* **Legal clauses:**  Standard contractual terms and conditions.


**Conclusion:**

Siemens demonstrates strong potential to meet many of the technical requirements of the Barclayville Solar Power Plant tender based on the limited information provided. However, a complete evaluation requires detailed specifications for all proposed equipment and a thorough assessment of the tender's complete commercial terms.  The "Partially Compliant" markings highlight the need for a detailed proposal that clearly addresses each requirement to achieve full compliance.


In [26]:
# add the last response to the chat history
add_history_to_chat_single({'prompt': prompt_clause_by_clause, 'answer': response_clause_by_clause.text}, "technical engineer", history_chat)

## Count the overall tokens

The total number of token can be computed by counting the tokens of history_chat, since the new responses have been appended to it for each call of Gemini.

In [27]:
print(f"{model.count_tokens(history_chat)=}")

model.count_tokens(history_chat)=total_tokens: 666545



# Alternative usage of Gemini: Context caching

As the entire dataset consists in JSON files, it could be cached using the Context caching functionality of Gemini:

https://ai.google.dev/gemini-api/docs/caching?lang=python

With context caching the system stores intermediate results from prior tender evaluations or company analyses, so if a similar query arises, the system can quickly retrieve relevant data and produce faster, more accurate responses.

In [28]:
# to ensure that no caching limit is exceeded, flush all the already available caches
from google.generativeai import caching

def delete_caches() -> None:
    for c in caching.CachedContent.list():
        print(f"Deleting cache named \"{c.display_name}\" ...")
        c.delete()
    print("All the caches have been deleted!")

delete_caches()

All the caches have been deleted!


In [29]:
import google.generativeai as genai
import time

@retry_on_failure(wait_time_seconds=10, max_retries=2)
def process_file_for_caching(file_path: str) -> genai.types.file_types.File | None:
    if os.path.exists(file_path):
        loaded_file = genai.upload_file(file_path)
        
        while loaded_file.state.name == "PROCESSING":
            print(f"Processing file {file_path} ...")
            time.sleep(2)
            loaded_file = genai.get_file(loaded_file.name)
        print(f"Processing file {file_path} completed. Available at {loaded_file.uri}")
        
        return loaded_file
    else:
        return None

# the process_file_for_caching() function will make the input file as available for caching
tenders_info_file = process_file_for_caching(tenders_info_json_path)
companies_info_file = process_file_for_caching(companies_info_json_path)

Processing file /kaggle/input/tenders-and-companies-websites/tenders_info.json completed. Available at https://generativelanguage.googleapis.com/v1beta/files/psmz8s3yyloj
Processing file /kaggle/input/tenders-and-companies-websites/companies_info.json completed. Available at https://generativelanguage.googleapis.com/v1beta/files/q1pnass7yv43


In [30]:
cache_instructions = f"""
{system_prompt}
The information you need is in the JSON files you have access to.
"""

In [31]:
import datetime

@retry_on_failure(wait_time_seconds=60, max_retries=2)
def build_cache_from_contents(model_name: str ='gemini-1.5-flash-002', cache_name: str='cache', instructions: str="Use the information to answer", resources: list=[], minutes_available: int=10):
    cache = caching.CachedContent.create(
        model=model_name,
        display_name=cache_name, # used to identify the cache
        system_instruction=(instructions),
        contents=resources,
        ttl=datetime.timedelta(minutes=minutes_available),
    )
    
    model_with_cache = genai.GenerativeModel.from_cached_content(cached_content=cache)
    return model_with_cache

files_to_cache = [tenders_info_file, companies_info_file]
model_with_cache = build_cache_from_contents(cache_name='tenders and companies info', instructions=cache_instructions, resources=files_to_cache)

In [32]:
def list_available_caches():
    cache_list = list(caching.CachedContent.list())
    if len(cache_list) == 0:
        print("Empty cache!")
        return []
        
    for c in cache_list:
        print(f"Available cache with name \"{c.display_name}\"\n    model: {c.model}\n    created: {c.create_time}\n    expires: {c.expire_time}\n    tokens: {c.usage_metadata.total_token_count}")
        # print(c)
    return cache_list

caches = list_available_caches()

Available cache with name "tenders and companies info"
    model: models/gemini-1.5-flash-002
    created: 2024-11-26 14:27:02.623039+00:00
    expires: 2024-11-26 14:37:02.153318+00:00
    tokens: 668315


## Test if the cache works

In [33]:
def ask_gemini_with_cache_dummy_response(model, prompt: str=""):
    """
    Function to handle a possible error by Gemin when generating a response
    """
    try:
        response = model.generate_content(prompt)
    except Exception as exc:
        print(f"Execution in model with caching failed: {exc}")
        print("An empty response will be returned")
        
        class DummyResponse:
            def __init__(self):
                self.text = "MODEL WITH CACHING FAILED TO GENERATE RESPONSE"
        response = DummyResponse()
        
    return response

@retry_on_failure(wait_time_seconds=2, max_retries=5)
def ask_gemini_with_cache(model, prompt: str=""):
    """
    Wrapper for a Gemini model that uses cache
    """
    response = model.generate_content(prompt)
    return response

In [34]:
response_cached_files = ask_gemini_with_cache_dummy_response(model=model_with_cache, prompt="Describe the documents you have access to")

Markdown(response_cached_files.text)

Execution in model with caching failed: 500 Internal error encountered.
An empty response will be returned


MODEL WITH CACHING FAILED TO GENERATE RESPONSE

## Generate the clause-by-clause with cached content

In [35]:
response_cached_clause_by_clause = ask_gemini_with_cache_dummy_response(model=model_with_cache, prompt=f"Use the documents you have access to, to answer.\n {user_prompt}")

Markdown(response_cached_clause_by_clause.text)

Execution in model with caching failed: 500 Internal error encountered.
An empty response will be returned


MODEL WITH CACHING FAILED TO GENERATE RESPONSE

## Deleting the generated caches

In [36]:
delete_caches()

Deleting cache named "tenders and companies info" ...
All the caches have been deleted!
